<a href="https://colab.research.google.com/github/menasiraziz/Convnet/blob/master/convnet3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

In [0]:
from numpy.lib.stride_tricks import as_strided


def strided_convolution3D(image, weight, stride):
    m,ch,im_h, im_w = image.shape
    f,t,f_h, f_w = weight.shape
    out_shape = (m,ch,1 + (im_h - f_h) // stride, 1 + (im_w - f_w) // stride, f_h, f_w)
    out_strides = (image.strides[0],image.strides[1],image.strides[2] * stride, image.strides[3] * stride, image.strides[2], image.strides[3])
    windows = as_strided(image, shape=out_shape, strides=out_strides)
    return np.einsum('mcopjk,ecjk->meop',windows,weight)

def mx_pool(img,s): # function takes advatange of row order of numpy array
  m,ch,n,n=img.shape # flatten all images into stride 
  c=img.reshape(-1,s) #change s to change horizontal stride
  ind=np.argmax(c,axis=1)
  ind1=np.ravel_multi_index([np.arange(ind.shape[0]),ind], (ind.shape[0],s))
  d=img.flatten()[ind1]
  g=d.reshape(-1,s,int(n/s)) #probably i need to change s in the middle to change vertical stride

  h=g.argmax(axis=1)
  i1=np.arange(g.shape[0]) # varies along images in data 
  i2=np.arange(g.shape[2]) # varies along col axis in indices
  ss=i1[:,np.newaxis]*n+g.shape[2]*h+i2 # i2 is varying along col and i1 is varying along batch and row dim
  return ind1[ss.flatten()]


import numpy as np
from numpy.lib.stride_tricks import as_strided


def strided_convolution3D1_g(image, weight, stride):
    m,ch,im_h, im_w = image.shape
    f,t,f_h, f_w = weight.shape
    #print(m)
    out_shape = (m,ch,1 + (im_h - f_h) // stride, 1 + (im_w - f_w) // stride, f_h, f_w)
    out_strides = (image.strides[0],image.strides[1],image.strides[2] * stride, image.strides[3] * stride, image.strides[2], image.strides[3])
    windows = as_strided(image, shape=out_shape, strides=out_strides)
    #print(windows.shape)
    #print(windows)
    return np.einsum('meopjk,ecjk->mecop',windows,weight)
    #return np.einsum('meopjk,ecjk->meop',windows,weight)
    #return np.einsum('meopjk,ecjk->mecop',windows,weight)

def strided_convolution3D_grad1(image, weight, stride):
    m,ch,im_h, im_w = image.shape
    m1,m2,f_h, f_w = weight.shape
    
    #print(m)
    out_shape = (m,ch,1 + (im_h - f_h) // stride, 1 + (im_w - f_w) // stride, f_h, f_w)
    out_strides = (image.strides[0],image.strides[1],image.strides[2] * stride, image.strides[3] * stride, image.strides[2], image.strides[3])
    windows = as_strided(image, shape=out_shape, strides=out_strides)
    #print(windows.shape)
    #print(windows[0])
    #print(windows*gg1)
    return np.einsum('mcopjk,mejk->mecop',windows,weight)


In [0]:
class ConvLayer:
  def __init__(self,in_ch=1,out_ch=1,kernel=(2,2),stride=1):
    self.filters=np.random.randn(out_ch,in_ch,kernel[0],kernel[1])
    self.stride=stride

  def feedforward(self,x):
    if(x.shape[1]!=self.filters.shape[1]):
      print(x.shape[1])
      print(self.filters.shape[1])
      raise Exception("channels in input and output are not same")
    self.out=strided_convolution3D(x,self.filters,self.stride)
    return self.out
  def grad(self,x,loss_grad):
    self.df=strided_convolution3D_grad1(x,loss_grad,1).sum(axis=0)
    gg=np.rot90(loss_grad,2,axes=(2,3))
    gg1=np.pad(gg, ((0, 0),(0, 0),(1, 1),(1,1)), 'constant', constant_values=(0))
    ooo=strided_convolution3D1_g(gg1,self.filters,1)
    return np.rot90(ooo,2,axes=(3,4)).sum(axis=1)

class FC_Layer:
  def __init__(self,i_dim,out_dim):
    self.W=np.random.randn(i_dim,out_dim)
    self.dw=np.zeros(self.W.shape)
  def feedforward(self,x):
    self.out=np.dot(x,self.W)
    return self.out
  def grad(self,x,loss_grad):
    return self.W*loss_grad[:,np.newaxis]

class Network:
  def __init__(self):
    self.n=7
    self.in_ch=1
    self.m=1
    self.f1=2
    self.f2=2
    self.cs=1
    self.o_ch=1

    self.mxs=2

    self.conv1=ConvLayer(in_ch=self.in_ch,out_ch=self.o_ch,kernel=(self.f1,self.f1),stride=self.cs)

    self.conv2=ConvLayer(in_ch=1,out_ch=1,kernel=(self.f2,self.f2),stride=self.cs)

    cos=(self.n-self.f1+1)//self.mxs-self.f2+1

    self.FC1=FC_Layer(cos*cos,1)

  def gen_images(self):
    return np.random.randn(self.m,self.in_ch,self.n,self.n)

  def feedforward(self,x):
    c1=self.conv1.feedforward(x)
    self.m1_indices=mx_pool(c1,self.mxs)
    self.m1=c1.flatten()[self.m1_indices].reshape(-1,c1.shape[1],int(c1.shape[2]/self.mxs),int(c1.shape[3]/self.mxs))
    c2=self.conv2.feedforward(self.m1)
    f1=self.FC1.feedforward(c2.reshape(c2.shape[0],-1))
    return f1
  def backpropagate(self,x,y,yhat):
    loss_grad=2*(yhat-y)
    gradF1=self.FC1.grad(self.conv2.out.reshape(loss_grad.shape[0],-1),loss_grad).reshape(self.conv2.out.shape)
    print(gradF1.shape)
    gradC2=self.conv2.grad(self.m1,gradF1)
    print(gradC2.shape)
    grad_zeros=np.zeros(self.conv1.out.shape[0]*self.conv1.out.shape[1]*self.conv1.out.shape[2]*self.conv1.out.shape[3])
    grad_zeros[self.m1_indices]=gradC2.flatten()
    gg=grad_zeros.reshape(self.conv1.out.shape)
    gradC1=self.conv1.grad(x,gg)
    print(gradC1)

  def loss(self,y,yhat):
    return np.sum(np.square(y-yhat))

  def num_grad(self,x,y):
    ep=np.zeros(x.shape)
    dw=np.zeros(x.shape)
    for dd in range(x.shape[0]):
      for k in range(x.shape[1]):
       for i in range(x.shape[2]):
         for j in range(x.shape[3]):

           ep[dd,k,i,j]=1e-4
           yhat1=self.feedforward(x+ep)
           yhat2=self.feedforward(x-ep)
           #print(yhat1.shape)
           dw[dd,k,i,j]=(self.loss(y,yhat1)-self.loss(y,yhat2))/2e-4
           ep[dd,k,i,j]=0

    return dw

In [24]:
np.random.seed(100)
nn=Network()
img=nn.gen_images()*10
y=np.random.randn(img.shape[0],1)
yhat=nn.feedforward(img)
nn.backpropagate(img,y,yhat)
nn.num_grad(img,y)

(1, 1, 2, 2)
(1, 1, 3, 3)
[[[[  0.           0.          -6.03819631   1.18254222   0.
      0.           0.        ]
   [  7.3486271   -1.43918173   3.97896555  -0.87112151  -5.18186599
      1.01483539   0.        ]
   [ 14.57604583  -2.74282188   0.         -17.80994748   6.90263763
     -0.74758002   0.        ]
   [-12.79615633   2.80148368   0.          11.7361483   -0.6293415
     -0.37995089   0.        ]
   [  0.           0.           0.           0.          -1.27844189
     18.68122784  -3.60378433]
   [  0.           4.00342616  -0.78404547 -23.17172868   4.53803521
    -12.12585347   2.65473317]
   [  0.          -2.63812137   0.57756828  15.26937935  -3.34295049
      0.           0.        ]]]]


array([[[[  0.        ,   0.        ,  -6.03819631,   1.18254222,
            0.        ,   0.        ,   0.        ],
         [  7.3486271 ,  -1.43918173,   3.97896555,  -0.87112151,
           -5.18186599,   1.01483539,   0.        ],
         [ 14.57604583,  -2.74282188,   0.        , -17.80994748,
            6.90263763,  -0.74758002,   0.        ],
         [-12.79615633,   2.80148368,   0.        ,  11.7361483 ,
           -0.6293415 ,  -0.37995089,   0.        ],
         [  0.        ,   0.        ,   0.        ,   0.        ,
           -1.27844189,  18.68122784,  -3.60378433],
         [  0.        ,   4.00342616,  -0.78404547, -23.17172868,
            4.53803521, -12.12585347,   2.65473317],
         [  0.        ,  -2.63812137,   0.57756828,  15.26937935,
           -3.34295049,   0.        ,   0.        ]]]])

In [0]:
np.random.seed(0)
n=5
lay=FC_Layer(n,1)
m=3
x=np.random.randn(m,n)
y=np.random.randn(m,1)
yhat=lay.feedforward(x)
loss_grad=2*(yhat-y)
lay.grad(x,loss_grad).reshape(-1,n)

In [0]:
loss_grad

array([[5.45294691],
       [4.98561353],
       [2.94279091]])

In [0]:
ll=lay.W*loss_grad[:,np.newaxis]
ll.flatten()

array([9.61928379, 2.18203601, 8.79488324, 1.99502919, 5.1912372 ,
       1.17757899])

In [0]:
lay.W*loss_grad[0]

array([[9.61928379],
       [2.18203601]])

In [0]:
class ConvLayer:
  def __init__(self,in_ch=1,out_ch=1,kernel=(2,2),stride=1):
    self.filters=np.random.randn(out_ch,in_ch,kernel[0],kernel[1])
    self.stride=stride

  def feedforward(self,x):
    if(x.shape[1]!=self.filters.shape[1]):
      print(x.shape[1])
      print(self.filters.shape[1])
      raise Exception("channels in input and output are not same")
    self.out=strided_convolution3D(x,self.filters,self.stride)
    return self.out
  def grad(self,x,loss_grad):
    self.df=strided_convolution3D_grad1(x,loss_grad,1).sum(axis=0)
    gg=np.rot90(loss_grad,2,axes=(2,3))
    gg1=np.pad(gg, ((0, 0),(0, 0),(1, 1),(1,1)), 'constant', constant_values=(0))
    ooo=strided_convolution3D1_g(gg1,self.filters,1)
    return np.rot90(ooo,2,axes=(3,4)).sum(axis=1)
  def perturb_f(self,ep):
    self.filters=self.filters+ep


class Max_Pool_Layer:
  def __init__(self,stride):
    self.stride=stride
  def feedforward(self,x):
    self.m_indices=mx_pool(x,self.stride)
    self.m=x.flatten()[self.m_indices].reshape(-1,x.shape[1],
                                               int(x.shape[2]/self.stride),int(x.shape[3]/self.stride))
    return self.m
  def grad(self,x,grad_loss):
    grad_zeros=np.zeros(x.shape[0]*x.shape[1]*x.shape[2]*x.shape[3])
    grad_zeros[self.m_indices]=grad_loss.flatten()
    gg=grad_zeros.reshape(x.shape)
    return gg


class FC_Layer:
  def __init__(self,i_dim,out_dim):
    self.W=np.random.randn(i_dim,out_dim)
    self.dw=np.zeros(self.W.shape)
  def feedforward(self,x):
    self.out=np.dot(x,self.W)
    return self.out
  def grad(self,x,loss_grad):
    return self.W*loss_grad[:,np.newaxis]

class Network:
  def __init__(self):
    self.n=12
    self.in_ch=2
    self.m=10
    self.f1=3
    self.f2=2
    self.cs=1
    self.o_ch=2

    self.mxs=2

    self.conv1=ConvLayer(in_ch=self.in_ch,out_ch=self.o_ch,kernel=(self.f1,self.f1),stride=self.cs)

    self.conv2=ConvLayer(in_ch=2,out_ch=1,kernel=(self.f2,self.f2),stride=self.cs)

    self.mxp1=Max_Pool_Layer(self.mxs)

    cos=(self.n-self.f1+1)//self.mxs-self.f2+1

    self.FC1=FC_Layer(cos*cos,1)

  def gen_images(self):
    return np.random.randn(self.m,self.in_ch,self.n,self.n)

  def feedforward(self,x):
    c1=self.conv1.feedforward(x)
    m1=self.mxp1.feedforward(c1)
    c2=self.conv2.feedforward(m1)
    f1=self.FC1.feedforward(c2.reshape(c2.shape[0],-1))
    return f1
  def backpropagate(self,x,y,yhat):
    loss_grad=2*(yhat-y)
    gradF1=self.FC1.grad(self.conv2.out.reshape(loss_grad.shape[0],-1),loss_grad).reshape(self.conv2.out.shape)
    #print(gradF1.shape)
    gradC2=self.conv2.grad(self.mxp1.m,gradF1)
    #print(gradC2.shape)
    grad_mxp1=self.mxp1.grad(self.conv1.out,gradC2)
    gradC1=self.conv1.grad(x,grad_mxp1)
    #print(gradC1)

  def loss(self,y,yhat):
    return np.sum(np.square(y-yhat))

  def num_grad(self,x,y):
    ep=np.zeros(x.shape)
    dw=np.zeros(x.shape)
    for dd in range(x.shape[0]):
      for k in range(x.shape[1]):
       for i in range(x.shape[2]):
         for j in range(x.shape[3]):

           ep[dd,k,i,j]=1e-4
           yhat1=self.feedforward(x+ep)
           yhat2=self.feedforward(x-ep)
           #print(yhat1.shape)
           dw[dd,k,i,j]=(self.loss(y,yhat1)-self.loss(y,yhat2))/2e-4
           ep[dd,k,i,j]=0

    return dw

  def num_grad_df(self,x,y,shape,func):
    #f=self.conv2.filters
    #func=self.conv2.perturb_f
    ep=np.zeros(shape)
    dw=np.zeros(shape)
    #print(self.conv2.filters)
    for dd in range(shape[0]):
      for k in range(shape[1]):
       for i in range(shape[2]):
         for j in range(shape[3]):

           ep[dd,k,i,j]=1e-4
           func(ep)
           #print(self.conv2.filters)
           yhat1=self.feedforward(x)
           func(-2*ep)
           yhat2=self.feedforward(x)
           #print(self.conv2.filters)
           func(ep)
           #print(yhat1.shape)
           dw[dd,k,i,j]=(self.loss(y,yhat1)-self.loss(y,yhat2))/2e-4
           ep[dd,k,i,j]=0

    return dw

In [72]:
np.random.seed(100)
nn=Network()
img=nn.gen_images()*10
y=np.random.randn(img.shape[0],1)
yhat=nn.feedforward(img)
nn.backpropagate(img,y,yhat)
#nn.num_grad(img,y)
print(nn.conv1.df)
nn.num_grad_df(img,y,nn.conv1.filters.shape,nn.conv1.perturb_f)

[[[[-1647226.65336475  -500902.02328258   846200.26792343]
   [ -579101.24084628   863026.29740094   492295.56445153]
   [  189301.57272224 -1086762.09144228  -737160.67987375]]

  [[  938565.71598106  -334080.03171034  1060481.80275155]
   [ -211906.06289044  3194256.63685154  1063801.93299418]
   [ 1365172.63262884   319152.54777431  1056268.09502134]]]


 [[[ -417009.75933885  -642843.20982547   328638.7936571 ]
   [  481879.54290746  -387757.37856268  -542231.24471764]
   [ -318771.32715056   368763.82471907   135132.8771429 ]]

  [[  603316.64427842   -82222.70565587   271685.5415042 ]
   [  613111.99023078  -793781.28773355   235643.78667596]
   [  390771.17595606   390919.57524453   203022.54567157]]]]


array([[[[-1647226.65335983,  -500902.0232968 ,   846200.267924  ],
         [ -579101.24083981,   863026.29740909,   492295.56445032],
         [  189301.57271214, -1086762.09143363,  -737160.67988425]],

        [[  938565.71597978,  -334080.03170975,  1060481.80275597],
         [ -211906.06288612,  3194256.63684495,  1063801.93298683],
         [ 1365172.63262533,   319152.54778229,  1056268.09502952]]],


       [[[ -417009.75934975,  -642843.20982173,   328638.79366778],
         [  481879.5429077 ,  -387757.37856515,  -542231.24471493],
         [ -318771.32715657,   368763.82472925,   135132.87714683]],

        [[  603316.64427184,   -82222.70565107,   271685.54149568],
         [  613111.99022457,  -793781.2877167 ,   235643.78667623],
         [  390771.17595822,   390919.57524419,   203022.54566923]]]])

In [37]:
nn.conv2.filters.shape

(1, 1, 2, 2)

In [29]:
ftft=nn.conv1.filters
print(ftft)
ftft[0]=0
print(ftft)
print(nn.conv1.filters)

[[[[-1.74976547  0.3426804 ]
   [ 1.1530358  -0.25243604]]]]
[[[[0. 0.]
   [0. 0.]]]]
[[[[0. 0.]
   [0. 0.]]]]
